In [1]:
def ecg_segmented_beat_modulation_noise_removal(ecg, fs, r_peaks, delta_t=40e-3):
    """
    ECG artifact removal using segmented-beat modulation.
    Full credit goes to the authors of the paper.
    
    Parameters
    ----------
    ecg: type
        Original (pre-processed) ECG signal [uV]
        
    fs:
        Sampling rate [Hz]
        
    r_peaks:
        Location of R peaks [samples]
        
    delta_t:
        Half duration of the QRS segment [s]
        Default value is 40e-3
    
    Returns
    -------
    ecg_clean:
        Clean ECG signal [uV]
        
    mCC:
        Median of the cardiac cycle [uV]
        
    cc_modulated:
        Modulated cardiac cycles [uV]
        
    References
    ----------
    Agostinelli, Angela, Corrado Giuliani, and Laura Burattini. "Extracting a 
    clean ECG from a noisy recording: a new method based on segmented-beat 
    modulation." Computing in Cardiology Conference (CinC), 2014. IEEE, 2014.
    https://ieeexplore.ieee.org/abstract/document/7042976
    """

    # Important parameters.
    delta_t_n = delta_t * fs
    
    
    #%% Median cardiac cycle duration (mCCd) computation.
    
    # Left side of the block diagram of Fig. 2.
    n_beats = len(r_peaks)
                
    # Compute mCCd.
    # mCCd is the median duration of the cardiac cycles.
    # It is computed from the R-R intervals.
    rr_intervals = np.diff(r_peaks)
    mCCd = round(np.median(rr_intervals)) # [samples]
    
    # Compute the median duration of TUP cycles (mTUPd).
    mTUPd = mCCd - 2*delta_t_n # Median length of TUP segment [samples]

    
    #%% Median cardiac cycle (mCC) computation
    # Right side of the block diagram of Fig. 2.
    # (Beginning of each) cardiac cycle identification.
    cc = r_peaks - delta_t_n
    
    # Cast to int.
    cc = cc.astype(int)
    
    # Cardiac cycle segmentation.
    # Notice that we won't take into account the last cardiac cycle, since it
    # might very well be incomplete.
    # With further processing, the last cycle could be used, filling the
    # missing data with NaN (for instance).
    # The more cardiac cycles that are included, the smoother the ECG
    # signal will result.
    cc_segmentation = list()
    qrs_segments = list()
    tup_segments = list()
    tup_segments_modulated = list()
    cc_modulated = list()
    for ii in range(0, n_beats-1):
        cc_segmentation.append(ecg[cc[ii]:cc[ii+1]])
        qrs_segments.append(cc_segmentation[ii][0:int(delta_t_n*2)])
        tup_segments.append(cc_segmentation[ii][int(delta_t_n*2):])

        # Modulate TUP segments.
        tup_segments_modulated.append(signal.resample(tup_segments[ii], int(mTUPd)))
        
        # Cardiac cycle reconstruction.
        cc_modulated.append(np.concatenate((qrs_segments[ii], tup_segments_modulated[ii])))
        

    # Compute mCC (median of the cardiac cycle).
    # The original paper uses the median.
    # If wished, this could be replaced by the mean.
    mCC = np.median(cc_modulated, 0)
     
    #%% Clean ECG extraction (Fig. 4)
    
    # mCC segmentation.
    qrs_segment_median = mCC[0:int(delta_t_n*2)]
    tup_segment_median = mCC[int(delta_t_n*2):]
    
    # Demodulate tup_segment_median.
    tup_segments_demodulated = list()
    cc_demodulated = list()
    for ii in range(0, n_beats-1):
        tup_segments_demodulated.append(signal.resample(tup_segment_median, len(tup_segments[ii])))
        cc_demodulated.append(np.concatenate((qrs_segment_median, tup_segments_demodulated[ii])))


    # Cardiac cycle concatenation.
    ecg_clean = [item for sublist in cc_demodulated for item in sublist]

    return (ecg_clean, mCC, cc_modulated)

In [5]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import os
import pickle

#Change this for each participant
part_number = '203'

# Set matplotlib parameters for displaying graphs
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = [9, 4.5]  # Bigger images
plt.rcParams['font.size']= 14
pd.set_option('display.max_columns', None)

save_dir = '../data/part'+part_number+'/figures/'

p = '../data/part'+part_number+'/part'+part_number+'_pilot_ecg_signals.p'
ecg = pickle.load(open(p,"rb"))
eda = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_eda_signals.p',"rb"))
rsp = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_rsp_signals.p',"rb"))
baseline_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_ecg_signals.p'
baseline2_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_eda_signals.p'
baseline3_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_rsp_signals.p'
ecg_baseline = pickle.load(open(baseline_fn,"rb"))
eda_baseline = pickle.load(open(baseline2_fn,"rb"))
rsp_baseline = pickle.load(open(baseline3_fn,"rb"))

data_file = '../data/part'+part_number+'/part'+part_number+'_pilot.acq'
rate = 2000

data, sampling_rate = nk.read_acqknowledge(data_file)
data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                            "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})

timestamps=np.loadtxt('../support/trigger timestamps/trigger timestamps.csv',dtype='int',delimiter=',')
start_times = timestamps[int(part_number[-2:]) - 1]
feedback_times = timestamps[int(part_number[-2:]) - 1 + 15]

taskloads_all = np.loadtxt('../support/taskload settings/taskload settings.csv',dtype='int',delimiter=',',encoding='UTF-8')
taskload_settings = taskloads_all[int(part_number[-2:]) - 1]

durations = [100,100,100,100,100,100,100,100,100,100,100,100]
#durations = [633339, 472624, 462436, 479671, 443789, 460186, 451913, 467455, 465081, 407503, 453908, 453551]

events = {}
events['onset'] = start_times
events['duration'] = durations
events['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
events['condition'] =  taskload_settings
#events['trust'] = trust_settings DOES NOT WORK!

events_feedback = {}
events_feedback['onset'] = feedback_times
events_feedback['duration'] = durations
events_feedback['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
events_feedback['condition'] =  taskload_settings

ecg_pilot_interval = nk.epochs_create(ecg, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
eda_pilot_interval = nk.epochs_create(eda, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
rsp_pilot_interval = nk.epochs_create(rsp, events, sampling_rate=rate, epochs_start=0, epochs_end=50)

ecg_trust_interval = nk.epochs_create(ecg,events,sampling_rate=rate,epochs_start=50,epochs_end=70)
eda_trust_interval = nk.epochs_create(eda,events,sampling_rate=rate,epochs_start=50,epochs_end=70)
rsp_trust_interval = nk.epochs_create(rsp,events,sampling_rate=rate,epochs_start=50,epochs_end=70)

ecg_trust_event = nk.epochs_create(ecg,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)
eda_trust_event = nk.epochs_create(eda,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)
rsp_trust_event = nk.epochs_create(rsp,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)

pi_high_tl = {}
hc=1
pi_med_tl = {}
mc=1
pi_low_tl = {}
lc=1
for i in range(1,13):
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = ecg_pilot_interval[i]
        hc+=1
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = ecg_pilot_interval[i]
        mc+=1
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = ecg_pilot_interval[i]
        lc+=1

for i in range(1,13):
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = eda_pilot_interval[i]
        hc+=1
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = eda_pilot_interval[i]
        mc+=1
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = eda_pilot_interval[i]
        lc+=1
            
for i in range(1,13):
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = rsp_pilot_interval[i]
        hc+=1
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = rsp_pilot_interval[i]
        mc+=1
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = rsp_pilot_interval[i]
        lc+=1
        
x_axis_pilot = np.linspace(0, ecg_pilot_interval[1].shape[0]/ sampling_rate, ecg_pilot_interval[1].shape[0])

In [6]:

r_peaks = ecg_pilot_interval[1]['ECG_R_Peaks'][:]

In [9]:
(ecg_clean, mCC, cc_modulated) = ecg_segmented_beat_modulation_noise_removal(ecg_pilot_interval[1]['ECG_Raw'],
                                                                            2000, r_peaks)

KeyError: 1